# 프로젝트 3-1 : 신경망 정확성 향상 시키기

## 과적합(Overfit), 드롭아웃(Drop Out), 그리고 정규화(Regularization)

- 조기 종료 : 과적합 & 검증오차
- 학습 진행하면 학습 오차는 꾸준히 내려가 0으로 수렴
- 검증 오차는 어느 순간 올라가기 시작함
- 조기 종료 : 검증 오차가 올라가는 순간 포착하여 학습 종료하는 것

- 과적합 방지 : 학습데이터를 늘림, 드롭아웃 모델에 적용

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [3]:
EPOCHS = 50
BATCH_SIZE = 64

# 데이터 셋에 노이즈 추가하기

- transforms.RandomHorizontalFlip() : 무작위 수평뒤집기로 학습 예제 2배로 늘리기 가능


In [5]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./.data',
                      train=True,
                      download=True,
                      transform=transforms.Compose([
                          transforms.RandomHorizontalFlip(),
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))
                          
                      ])),
        batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./.data',
                      train=False,
                      transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))
                      ])),
        batch_size=BATCH_SIZE, shuffle=True)

Extracting ./.data\MNIST\raw\train-images-idx3-ubyte.gz to ./.data\MNIST\raw


Extracting ./.data\MNIST\raw\train-labels-idx1-ubyte.gz to ./.data\MNIST\raw



Extracting ./.data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./.data\MNIST\raw


Extracting ./.data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data\MNIST\raw
Processing...


C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


# Fashion MNIST 학습하기

- dropout_p : 드롭아웃 비율 설정( 예, 0.2 = 20% 뉴런 사용 안함), 학습 모드에서만 DropOUt 실행
- 평가모드는 모든 뉴런 사용

In [7]:
class Net(nn.Module) :
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        # 드롭 아웃 혹률
        self.dropout_p = dropout_p
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        # 드롭 아웃 추가
        x = F.dropout(x, training = self.training,
                      p=self.dropout_p)
        x = F.relu(self.fc2(x))
        # 드롭 아웃 추가
        x = F.dropout(x, training = self.training,
                     p=self.dropout_p)
        x = self.fc3(x)
        return x

# 모델 설정

In [8]:
model = Net(dropout_p = 0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.01)

# 학습하기

In [9]:
def train(model, train_loader, optimizer) :
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader) :
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

# 테스트 하기

In [10]:
def evaluate(model, test_loader) :
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader :
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target,
                                        reduction = 'sum').item()
            
            #맞춘 개수 계산
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

# 실행

- 이전 모델(프로젝트3)과 정확성 및 오류 비교

In [ ]:
for epoch in range(1, EPOCHS + 1) :
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
           epoch, test_loss, test_accuracy))

[1] Test Loss: 0.4292, Accuracy: 86.22%
[2] Test Loss: 0.3509, Accuracy: 89.10%
[3] Test Loss: 0.2881, Accuracy: 91.13%
[4] Test Loss: 0.2480, Accuracy: 92.42%
[5] Test Loss: 0.2237, Accuracy: 93.15%
[6] Test Loss: 0.2048, Accuracy: 93.63%
[7] Test Loss: 0.1924, Accuracy: 94.11%
[8] Test Loss: 0.1793, Accuracy: 94.54%
[9] Test Loss: 0.1703, Accuracy: 94.84%
[10] Test Loss: 0.1609, Accuracy: 94.95%
[11] Test Loss: 0.1528, Accuracy: 95.28%
[12] Test Loss: 0.1457, Accuracy: 95.41%
[13] Test Loss: 0.1386, Accuracy: 95.69%
[14] Test Loss: 0.1370, Accuracy: 95.71%
[15] Test Loss: 0.1334, Accuracy: 95.91%
[16] Test Loss: 0.1293, Accuracy: 95.91%
[17] Test Loss: 0.1236, Accuracy: 96.08%
[18] Test Loss: 0.1222, Accuracy: 96.11%
[19] Test Loss: 0.1194, Accuracy: 96.17%
[20] Test Loss: 0.1166, Accuracy: 96.25%
[21] Test Loss: 0.1148, Accuracy: 96.46%
[22] Test Loss: 0.1140, Accuracy: 96.40%
[23] Test Loss: 0.1125, Accuracy: 96.45%
[24] Test Loss: 0.1105, Accuracy: 96.55%
[25] Test Loss: 0.1063, A